In [ ]:
from __future__ import print_function, division, absolute_import 
import colamatch as clm
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import h5py
from brainmap.transformation import transformation
%matplotlib notebook
import logging
import sys
logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format='%(asctime)s %(name)s [%(levelname)s]:%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

# Artificial Testset

## Define filenames and read images and pointsets

In [ ]:
s1 = 1
s2 = 2
img_fname1 = "../data/section_%s.tif"%(str(s1).zfill(4))
img_fname2 = "../data/section_%s.tif"%(str(s2).zfill(4)) 
points_fname1 = "../data/section_%s.landmarks.txt"%(str(s1).zfill(4))
points_fname2 = "../data/section_%s.landmarks.txt"%(str(s2).zfill(4))

In [ ]:
# read landmarks and images
pointlist1 = np.loadtxt(points_fname1).astype('int')
pointlist2 = np.loadtxt(points_fname2).astype('int')
print(len(pointlist1), len(pointlist2))

## Perform constellation matching

In [ ]:
num_samples = 10000
sampler1 = clm.RandomSampler(len(pointlist1), 4, num_samples)
sampler2 = clm.RandomSampler(len(pointlist2), 4, num_samples)
start = time.time()
matches = clm.match(pointlist1, pointlist2, sampler1, sampler2, 
                    radius=0.1, coordinate_weight=2, ransac_threshold=0.01)
print("runtime for num_samples=%s: %f" % (num_samples,time.time()-start))

In [ ]:
img1 = cv2.imread(img_fname1,0)
img2 = cv2.imread(img_fname2,0)
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(1,2)
axs[0].imshow(img1, cmap='gray')
axs[1].imshow(img2, cmap='gray')
axs[0].plot(pointlist1[:,0], pointlist1[:,1],'w+',ms=15)
axs[1].plot(pointlist2[:,0], pointlist2[:,1],'w+',ms=15)
axs[0].axis('off')
axs[1].axis('off')
for i,candidate in enumerate(matches): 
    axs[0].plot(candidate[0,0], candidate[0,1], c=cmap(i), marker='o')
    axs[1].plot(candidate[1,0], candidate[1,1], c=cmap(i), marker='o')
fig.show()

# Match real vessel detections

### Load detected (& preregistered) landmarks, ROI coordinates and images

In [ ]:
### PARAMETERS ###
brainid = "B20"
structure = "V1_l"
fixed = 1529
moving = 1530
numLandmarks = 100   # use only best x landmarks for constellation matching
use_preregistration = True

In [ ]:
# read detected landmarks in fixed and moving image
l_fixed = h5py.File("../data/{}_{}_{}_vessels.h5".format(brainid,structure,fixed))["data"][:numLandmarks,:2]
l_moving = h5py.File("../data/{}_{}_{}_vessels.h5".format(brainid,structure,moving))["data"][:numLandmarks,:2]
print(l_fixed.shape[0], l_moving.shape[0])
if use_preregistration:
    # transform moving landmarks with preregistration\n",	
    prereg_file = "../data/{}_{}_transformation.json".format(brainid,moving)
    prereg = transformation.Transformation.from_json(prereg_file)
    prereg_inverse = transformation.Transformation.from_json(prereg_file, inverse=True)
    l_moving = prereg.apply_to_coords(l_moving, 0.001, 0.001)

## Constellation Matching

### Parameters for constellation matching:
* **num_samples**: Number of hashes added to KDTree for fixed and moving landmarks (depends on #landmarks -> more samples needed for increasing number of landmarks)
* **lambda**: Weight for absolute landmark coordinates in hash code (should be >1; 2 works good)
* **ransac**: Hessian threshold for homography ransac on matched normalized landmark coordinates (0.01 works good)
* **radius**: Radius for finding similar hashes in KDTree, could be calculated in match()-function with $$radius = \sqrt{(\lambda*maxDist*scale)^2*3 + \Delta relativ\_quad\_positions^2 * 4}$$, where $scale$ = scale used in normalization of landmarks (known in match() function), $maxDist$ = maximal distance between corresponding pixels in fixed and moving image (after preregistration, ~250), $\Delta relativ\_quad\_positions$ = maximal allowed distance between cx, cy, dx and dy of similar quads (0.0015 seems to be a good value here). Is it necessary to adapt radius in relation to the ratio of #landmarks/num_samples?

In [ ]:
# constellation matching
num_samples = 50000
sampler1 = clm.RandomSampler(len(l_fixed), 4, num_samples)
sampler2 = clm.RandomSampler(len(l_moving), 4, num_samples)
start = time.time()
matches = clm.match(l_fixed, l_moving, sampler1, sampler2, 
                    radius=0.025, coordinate_weight=2, ransac_threshold=0.01) 
print("runtime for num_samples=%s: %f" % (num_samples,time.time()-start))
if use_preregistration and len(matches) > 0:
    # transform moving match coordinates back to original moving image space:
    matches_moving = prereg_inverse.apply_to_coords(matches[:,1], 0.001, 0.001)
    matches = np.hstack((matches[:,0], matches_moving)).reshape(-1,2,2)
print("Found %d matches"%len(matches))

### Plot matches

In [ ]:
# load images and ROIs for plotting
scale = 0.1
roifile = h5py.File("../data/{}_{}_rois.h5".format(brainid,structure))
roi_fixed = roifile["{}/roi".format(fixed)][:]
roi_moving = roifile["{}/roi".format(moving)][:]
roifile.close()
img_fixed = cv2.imread("../data/{}_{}_{}_scale0.1.tif".format(brainid,structure,fixed),0)
img_moving = cv2.imread("../data/{}_{}_{}_scale0.1.tif".format(brainid,structure,moving),0)

In [ ]:
# plot landmarks and matches
if use_preregistration:
    l_moving_orig = prereg_inverse.apply_to_coords(l_moving, 0.001, 0.001)
else:
    l_moving_orig = l_moving
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(2,1)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
# plot (landmarks - roi-offset) * scale according to downscaled images (0.1)
axs[0].plot((l_fixed[:,0]-roi_fixed[0,0])*scale, (l_fixed[:,1]-roi_fixed[0,1])*scale,'w+',ms=10)
axs[1].plot((l_moving_orig[:,0]-roi_moving[0,0])*scale, (l_moving_orig[:,1]-roi_moving[0,1])*scale,'w+',ms=10)
axs[0].axis('off')
axs[1].axis('off')
for i,match in enumerate(matches): 
    # plot matched coordinates - roi-offset * scale according to downscaled images (0.1)
    axs[0].plot((match[0,0]-roi_fixed[0,0])*scale, (match[0,1]-roi_fixed[0,1])*scale, c=cmap(i), marker='o')
    axs[1].plot((match[1,0]-roi_moving[0,0])*scale, (match[1,1]-roi_moving[0,1])*scale, c=cmap(i), marker='o')
fig.show()

## K-Nearest-Neighbor Matching

In [ ]:
from skimage.measure import ransac
from skimage.transform import AffineTransform
import math
def homography_ransac(matches, residual_threshold=155):
    print("mts vor RANSAC: {}".format(matches.shape))
    landmarks1, landmarks2 = matches[:,0], matches[:,1]
    findingPosSampleProbability = 0.999
    percentageOutliers = 0.997
    numIterations = int(math.ceil(math.log(1-findingPosSampleProbability)/math.log(1-(1-percentageOutliers))))
    model_robust, inliers = ransac((landmarks1, landmarks2), AffineTransform,
                                   min_samples=3, residual_threshold=residual_threshold, max_trials=numIterations)
    if inliers is None:
        print("Ransac found no inliers.")
        inliers = list(range(len(matches)))
    result = matches[inliers]
    print("mts nach RANSAC: {}".format(result.shape))
    return result

from sklearn.neighbors import KDTree
def knn_matching(l_fixed, l_moving, k=3):
    matches = []
    tree_fixed = KDTree(l_fixed[:,:2], leaf_size=2, metric='minkowski')
    for l in l_moving:
        match_indices = tree_fixed.query([l], k=k, return_distance=False)[0]
        for m in match_indices:
            matches.append([l_fixed[m],l])
    tree_moving = KDTree(l_moving[:,:2], leaf_size=2, metric='minkowski')
    for l in l_fixed:
        match_indices = tree_moving.query([l], k=k, return_distance=False)[0]
        for m in match_indices:
            matches.append([l, l_moving[m]])
    if len(matches) > 0:
        matches = np.unique(np.array(matches), axis=0)
    matches = homography_ransac(matches)
    return matches

In [ ]:
start = time.time()
matches_knn = knn_matching(l_fixed, l_moving, k=2)
if use_preregistration and len(matches) > 0:
    # transform moving match coordinates back to original moving image space:
    matches_knn_moving = prereg_inverse.apply_to_coords(matches_knn[:,1], 0.001, 0.001)
    matches_knn = np.hstack((matches_knn[:,0], matches_knn_moving)).reshape(-1,2,2)
print("Found {} matches.".format(len(matches_knn)))
print("Runtime:",time.time()-start)

In [ ]:
# plot landmarks and matches
if use_preregistration:
    l_moving_orig = prereg_inverse.apply_to_coords(l_moving, 0.001, 0.001)
else:
    l_moving_orig = l_moving
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(2,1)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
# plot (landmarks - roi-offset) * scale according to downscaled images (0.1)
axs[0].plot((l_fixed[:,0]-roi_fixed[0,0])*scale, (l_fixed[:,1]-roi_fixed[0,1])*scale,'w+',ms=10)
axs[1].plot((l_moving_orig[:,0]-roi_moving[0,0])*scale, (l_moving_orig[:,1]-roi_moving[0,1])*scale,'w+',ms=10)
axs[0].axis('off')
axs[1].axis('off')
for i,match in enumerate(matches_knn): 
    # plot matched coordinates - roi-offset * scale according to downscaled images (0.1)
    axs[0].plot((match[0,0]-roi_fixed[0,0])*scale, (match[0,1]-roi_fixed[0,1])*scale, c=cmap(i), marker='o')
    axs[1].plot((match[1,0]-roi_moving[0,0])*scale, (match[1,1]-roi_moving[0,1])*scale, c=cmap(i), marker='o')
fig.show()

## Evaluation

In [ ]:
from brainmap.structures.matchers import drawing
def evaluate_matches(landmarks, matches, filenames, outfilename, factor=2): 
    """
    Args: 
        landmarks (array-like): 2x[N/M]x2 list, containing detected LOCAL landmarks (x,y) of 
                        filenames[0] and filenames[1] in original spacing.
        matches (dict): List of matched landmark coordinates per matching method (key), 
                        must also contain key "manually" with manually clicked ground truth matches.
        filenames (array-like): List of the two ROI image filenames
        outfilename (str): Format output filename, will be filled with matching-method-name+"_transformed_evaluated"
    """
    # ground truth (slice1 is moving, slice0 fixed)
    gt_affine = transformation.get_affine_transformation(matches["manually"][1],matches["manually"][0]) 
    t = transformation.Transformation(filenames[0],filenames[1],"affine","pixel",gt_affine[:2],spacing=0.001)
    x1_truth = t.apply_to_coords(matches["manually"][1], 0.001, 0.001)
    dists = [np.sqrt(np.sum((matches["manually"][0][i] - x1_truth[i])**2)) for i in range(len(x1_truth))] 
    mu = np.mean(dists)
    sigma = np.std(dists)
    print("mean and std deviation of manual matches under ground truth transformation: {} {}".format(mu,sigma))
    
    # get number of possible correct matches
    landmarks1_transformed = t.apply_to_coords(landmarks[1], 0.001, 0.001)
    pm = 0
    for l0 in landmarks[0]: 
        pm += sum([1 for l1 in landmarks1_transformed if np.sqrt(np.sum((l0 - l1)**2))-mu<=factor*sigma])
    print("possible correct matches: {}".format(pm))

    # get number of (in)correct matches per method
    correct = {} # save lists of indices of correct matches
    print
    print("%-25s %15s %10s / %-15s %10s %10s" %("method","mean distance","correct",
                                                "found matches", "recall", "precision"))
    for key, value in matches.items():
        if key == "manually": 
            continue
        x0 = value[:,0]
        x1 = t.apply_to_coords(value[:,1], 0.001, 0.001)
        dists = [np.sqrt(np.sum((x0[i] - x1[i])**2)) for i in range(len(x0))] 
        corr = []
        for i,e in enumerate(dists): 
            if abs(e-mu) <= factor*sigma: corr.append(i)
        correct[key] = corr
        recall = len(corr)/float(pm) 
        precision = len(corr)/float(len(value))
        print("%-25s %15.2f %10d / %-15d %10.2f %10.2f"%(key, np.mean(dists), len(corr), len(value), 
                                                         recall, precision))
        # assuming that images have spacing of 0.01 mm and match coordinates 0.001 mm
        drawing.draw_matches(None, value[:,0]*0.1, value[:,1]*0.1, filenames[0], filenames[1], 
                             outfilename.format(key+"_transformed_evaluated"), bbox0=None, bbox1=None, 
                             thickness=15, transformation=t, orig_spacing=0.01, correct_indices=corr)

In [ ]:
# add manually clicked matches (and convert into local coordinates): 
# save local, not preregistered match coordinates in original spacing in all_matches:
all_matches = {}
f = h5py.File("../data/{}-{}_manually_scale0.1.h5".format(fixed, moving))
all_matches["manually"] = np.array([f["/matches/others"][:,:2],       
                                    f["/matches/others"][:,2:4]])/scale
all_matches["knn"] = np.hstack((matches_knn[:,0] - roi_fixed[0], 
                                matches_knn[:,1] - roi_moving[0])).reshape(-1,2,2)
all_matches["constellation"] = np.hstack((matches[:,0] - roi_fixed[0], 
                                          matches[:,1] - roi_moving[0])).reshape(-1,2,2)

# run evalutation
evaluate_matches([l_fixed-roi_fixed[0], l_moving_orig-roi_moving[0]], all_matches, 
                 ["../data/{}_{}_{}_scale0.1.tif".format(brainid,structure,fixed), 
                  "../data/{}_{}_{}_scale0.1.tif".format(brainid,structure,moving)], 
                 "../data/%s_%s_{}.tif"%(brainid, structure), factor=4)